### Webscraper for extracting newly published names, strains and accession numbers from the weeekly IJSEM email (saved in html). Script then compares the IJSEM names to the NCBI names and generates a report used for taxonomy updates. 
### Uses Natural Language processing combined with REGEX to extract information from text
### Notes on selenium. There is a lot of bad advise in stack overflow that refers to old versions of selenium. Need to refer to the offocial documentation https://selenium-python.readthedocs.io/ If you run out of space in your linux directory the script will crash. There is a cache file written by selenium that may then need to be deleted to get the chrome driver working again. Follow the path in the error message and delete that cache directory

In [ ]:
pip install "numpy<2.0" 

In [ ]:
#pip install spacy
#pip install -U pip setuptools wheel
#pip install -U spacy

### Install the model at the command lines
small model
python -m spacy download en_core_web_sm

medium model
python -m spacy download en_core_web_md

large model
python -m spacy download en_core_web_lg

### create training set https://spacy.io/usage/training

This annotator works better https://agateteam.org/spacynerannotate/ combined with the DocBin code to create the .spacy file. 

https://spacy.io/usage/training#quickstart

python -m spacy init fill-config ./base_config.cfg ./config.cfg

This NER annotator worked better than the one above. Used the DocBin technique to convert to .spacy file. Spacy training command with config.cfg then worked.

Config.cfg needs to be created for the spacy command line training. 

Follow the instructions in the quickstart for the base_config.cfg file. Selected OS and clicked NER

https://spacy.io/usage/training#quickstart

Run the following at command line to create the config.cfg. 

python -m spacy init fill-config ./base_config.cfg ./config.cfg

Once the config file was done and spacy file was created run the following at command line to train the model

To debug the data:
python -m spacy debug data config.cfg --paths.train ./train.spacy --paths.dev ./train.spacy

To debug the config file file:
python -m spacy train config.cfg --output ./output

or

python -m spacy train config.cfg 

train the model at command line
python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./train.spacy

creates two directories model-best and model-last. I used model-best 

nlp1 = spacy.load(r"./output/model-best")


Notes
python -m spacy init fill-config ./base_config.cfg ./config.cfg

python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./dev.spacy

https://spacy.io/usage/training#basics

In [ ]:
#Always needed
import spacy
nlp = spacy.load("en_core_web_md")

### Train spacy if not already done. Using output https://arunmozhi.in/ner-annotator/


In [ ]:
from spacy.tokens import DocBin
import pandas as pd
import json
import os

### train the model at command line
python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./dev.spacy 

### Main webscraper program start


In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
#PATH = "C:\Users\mcveigh\Documents\PythonPC\chrome.exe"
#driver = webdriver.Chrome(PATH)
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
import re
import os
import sys
import bs4
from bs4 import BeautifulSoup
import requests
import numpy as np
from datetime import datetime
from nameparser import HumanName
import spacy
#nlp = spacy.load("en_core_web_sm")
nlp = spacy.load("en_core_web_md")
from spacy.matcher import PhraseMatcher
from spacy import displacy

/home/mcveigh/jup/lib/python3.9/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/mcveigh/jup/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [ ]:
#test
text2 = ("The type strain is 2205BS29-5T (=LMG 33062T =KACC 23240T), which is isolated from a marine sponge, P. elegans, "
"collected from Beomseom in Jeju-Island, Republic of Korea."

"The DNA G+C content of strain 2205BS29-5T is 67.8%. The GenBank accession numbers for the 16S rRNA gene and" 
"whole-genome sequences of strain 2205BS29-5T are OQ569368 and JAVAMQ000000000, respectively.")

In [ ]:
#test but need this later
nlp1 = spacy.load(r"./output/model-best") #load the best model
doc = nlp1(text2)
from spacy import displacy
displacy.render(doc, style="ent")

### Find URLS from saved email in html - save as from outlook in htm format. URLs are extracted and saved as input for selenium

In [2]:
input = (r'IJSEMemail34.htm')
output = (r'NameCheckweek34.xlsx')
alldescriptions = (r'all_descriptions34')

In [3]:
print(input)

IJSEMemail34.htm


In [4]:
#with open ('IJSEMemail33.htm', encoding = 'unicode_escape') as f:
with open (input, encoding = 'unicode_escape') as f:
    content = f.read()
    soup = BeautifulSoup(content, 'html.parser')  
#print(soup.prettify())

In [5]:
urls = []
numberoflinks = 0
for url in soup.findAll( 'a', attrs={'href': re.compile("^https:")}):
    urls.append(url.get('href'))
    numberoflinks = numberoflinks + 1
    #print(url)
print(numberoflinks)
#print(urls)
#remove_list = ['TandC','doi.org', 'myaccount']
urls = [e for e in urls if "TandC" not in e and "doi.org" not in e and "myaccount" not in e and "join-the-society" not in e]


#print(urls)

24


In [ ]:
#def find_strains(description):
    #strains = []
#    nlp1 = spacy.load(r"./output/model-best") #load the best model
#    doc = nlp1(description)
#    for entity in doc.ents:
#        if entity.text not in strains:
#            entity.text.encode('ascii', 'ignore').decode('unicode_escape')
#            strains.append(entity.text)
#    return strains

In [6]:
def find_strains(description):
    """Find strains with spacy"""
    #print('Entering strain subroutine')
    doc = nlp(description)
    nlp1 = spacy.load(r"./output/model-best") #load the best model
    phrase_matcher = PhraseMatcher(nlp.vocab)
    phrases = ['strain', 'Strain', 'strains', 'Strains']
    patterns = [nlp(description) for description in phrases]
    phrase_matcher.add('STRAIN', None, *patterns)
               
    for sent in doc.sents:    
        for match_id, start, end in phrase_matcher(nlp(sent.text)):
            if nlp.vocab.strings[match_id] in ["STRAIN"]:
                print(sent.text)
                doc = nlp1(sent.text)
                displacy.render(doc, style="ent")
                for entity in doc.ents:
                    if entity.text not in strains:
                        entity.text.encode('ascii', 'ignore').decode('unicode_escape')
                        strains.append(entity.text)
    return strains

def remove_non_ascii(text):
    """Remove non-ASCII characters"""
    return ''.join(char for char in text if ord(char) < 128)




### Main body - Selenium to extract data from html
### https://nameparser.readthedocs.io/en/latest/index.html to find the last name

In [7]:
from selenium.webdriver.common.by import By
pub_df = pd.DataFrame(columns=['PublishedName', 'Accessions', 'Strains', 'Authority', 'DOI', 'URL'])
pd.set_option('display.max_columns', None)
combined_description = []
for url in urls:
    counter = 1
    strains = []
    accessions = []
    orgname = []
    doi = []
    author = []
    date = []
    year = []
    author_raw = []
    author = []
    name1 = []
    name2 = []
    author_count = []
    authority = []
    description = None
    description1 = []
    description2 = []
    snumber = []
    #scrape_with_selenium(url, counter)
    #scrape_with_beautifulsoup(url)
    options = Options()
    options.headless = False  # Set to True for headless mode
    driver = webdriver.Chrome(options=options)

    #Navigate to the webpage
    driver.get(url)

    #Allow time for dynamic content to load (you may need to use WebDriverWait for more robust waiting)
    time.sleep(3)
    
    html = driver.page_source
    soup = BeautifulSoup(html)
    
    for element in driver.find_elements(By.CLASS_NAME, "item-meta-data__item-title"):
        #print(element.text)
        title = element.text
        print(title)
        
    for element in driver.find_elements(By.PARTIAL_LINK_TEXT, "doi.org"):
        doi = element.text
        #print(doi)
    
    #find publication year
    for element in driver.find_elements(By.XPATH, "//*[@id='bellowheadercontainer']/main/div[2]/div/ul/li[3]/span/span[2]"):
        date = element.text
        year = date[-4:]
        #print(year)
    
    #find authors
    for element in driver.find_elements(By.XPATH, "//*[@id='bellowheadercontainer']/main/div[2]/div/ul/li[1]/span"):
        author_raw = element.text
        #print(author_raw)
        author = re.sub(r"[\d+]+",'',author_raw)
        author = re.sub(r"†",'',author)
        author = re.sub(r" and ",',',author)
        author = re.sub(r",,",',',author)  
        author = author.split(',')
        author[:] = [item for item in author if item != '']
        #print(author)
        author_count = len(author)
        #print(author_count)
        if author_count == 1:
            name1 = (str(author[0])) 
            name1 = HumanName(name1)
            authority = name1.last + ' ' +str(year)
            #print(authority)
        elif author_count ==2:
            name1 = (str(author[0]))
            name1 = HumanName(name1)
            name2 = (str(author[1]))
            name2 = HumanName(name2)
            authority = name1.last + ' and ' + name2.last + ' ' +str(year)
            #print(authority)
        else:
            name1 = (str(author[0]))
            name1 = HumanName(name1)
            authority = name1.last + ' et al. ' +str(year)
            #print(authority)
    
    #extract data from each species description
    for element in driver.find_elements(By.CSS_SELECTOR, "div.tl-main-part.title"): #finds section headers
        #print(element.text)
        counter += 1
        description = element.text
        #print(description)
        if "Description of" in description:
            strains = []
            #print('found', description)  
            #snumber = 's' + str(counter - 4) + '/p[3]'
            snumber = 's' + str(counter - 4)
            #print('snumber is', snumber)
            for element in driver.find_elements(By.ID, snumber):
                description = element.text
                cleaned_text = remove_non_ascii(description)
                combined_description.append(cleaned_text)
                #print(cleaned_text)
                #print(description)
                        
                #find the organism names 
                orgname = []
                match = [r'(\S+\s+){2}(?=sp. nov)', r'(\S+\s+){2}(?=nom. nov.)', r'(\S+\s+){2}(?=SP. NOV.)', r'(\S+\s+){4}(?=subsp. nov.)']
                regex = re.compile(r'\b(' + '|'.join(match) + r')\b')
                if description is not None:
                    orgname = [m.group() for m in regex.finditer(description)]
                    print('orgname', orgname)

                #find the accessions
                pattern = [r'[A-Z]{2}\d{6}', r'[A-Z]{4}\d{8}', r'([A-Z]+)(_[A-Z]+)\d{6}', r'[A-Z]{6}\d{9}']
                regex = re.compile(r'\b(' + '|'.join(pattern) + r')\b')
                if description is not None:
                    accessions = [m.group() for m in regex.finditer(description)]
                    print('accessions', accessions)
    
                #find the strains
                if description is not None:
                    find_strains(cleaned_text)
                    print('strain names', strains)
    
                #load data into pandas dataframe
                row_data = [orgname, accessions, strains, authority, doi, url]
                length = len(pub_df)
                pub_df.loc[length] = row_data
            print('BREAK1')

   
    for element in driver.find_elements(By.CLASS_NAME, "tl-lowest-section"): #finds section headers
        description1 = element.text
        outer_html = element.get_attribute("outerHTML")
        if "Description of" in description1:          
            #print(outer_html)
            spans = soup.findAll('span', attrs = {'class' : 'tl-lowest-section'})
            for span in spans:
                if "Description of" in span.text:   
                    #print (span.text)
                    outer_div_id = span.find_parent('div').get('id')
                    print(f"Outer div ID: {outer_div_id}, Text: {span.text}")
                    for element in driver.find_elements(By.ID, outer_div_id):
                        description = element.text
                        cleaned_text = remove_non_ascii(description)
                        combined_description.append(cleaned_text)
                        #print(cleaned_text)
                        #print(description)
                        
                    #find the organism names   
                    orgname = []
                    match = [r'(\S+\s+){2}(?=sp. nov)', r'(\S+\s+){2}(?=nom. nov.)', r'(\S+\s+){2}(?=SP. NOV.)', r'(\S+\s+){4}(?=subsp. nov.)']
                    regex = re.compile(r'\b(' + '|'.join(match) + r')\b')
                    if description is not None:
                        orgname = [m.group() for m in regex.finditer(description)]
                        print('orgname', orgname)

                    #find the accessions
                    pattern = [r'[A-Z]{2}\d{6}', r'[A-Z]{4}\d{8}', r'([A-Z]+)(_[A-Z]+)\d{6}', r'[A-Z]{6}\d{9}']
                    regex = re.compile(r'\b(' + '|'.join(pattern) + r')\b')
                    if description is not None:
                        accessions = [m.group() for m in regex.finditer(description)]
                        print('accessions', accessions)
    
                    #find the strains
                    strains = []
                    if description is not None:
                        find_strains(cleaned_text)
                        print('strain names', strains)
    
                    #load data into pandas dataframe
                    row_data = [orgname, accessions, strains, authority, doi, url]
                    length = len(pub_df)
                    pub_df.loc[length] = row_data
                    print('BREAK2')



        
#Close the browser window
    driver.quit()    

Definition of the rhizobial symbiovars caraganae, robiniae and sophorae within Mesorhizobium and albiziae within Neomesorhizobium
Thalassobellus suaedae gen. nov., sp. nov., a marine bacterium of the family Flavobacteriaceae isolated from a halophyte Suaeda japonica
Outer div ID: s6-1, Text: Description of Thalassobellus gen. nov.
orgname []
accessions []
strain names []
BREAK2
Outer div ID: s6-2, Text: Description of Thalassobellus suaedae sp. nov.
orgname ['Thalassobellus suaedae ']
accessions ['OR229715', 'OR652609', 'OR229716', 'OR652608', 'CP134536', 'CP134537']
N.L. gen. n. suaedae, of Suaeda, the generic name of the halophyte, referring to the isolation of the type strain from the halophyte Suaeda japonica).



/home/mcveigh/jup/lib/python3.9/site-packages/spacy/displacy/__init__.py:213: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


The utilization of d-maltose and mannitol as a sole carbon source is variable depending on strains.


The type strain HL-DH10T (=KCCM 90512T=JCM 36598T) and sister strain HL-DH14 (=KCCM 90513


The type strain HL-DH10T (=KCCM 90512T=JCM 36598T) and sister strain HL-DH14 (=KCCM 90513


...

strain names ['BJSWXB5TM1T', 'BJSWXB5TM5T', 'GDMCC 1.4130T', 'KCTC 25759T']
orgname ['Streptococcus fermentans ']
accessions []
strain names ['BJSWXB5TM1T', 'BJSWXB5TM5T', 'GDMCC 1.4130T', 'KCTC 25759T']
BREAK1
Lacrimispora sinapis sp. nov., isolated from pickled potherb mustard (Brassica juncea Coss.)
orgname ['Lacrimispora sinapis ']
accessions ['PP065615', 'PP065615', 'CP141832', 'CP141832']
The type strain JR3T (=


The genomic DNA G+C content of strain JR3T is 44.0mol%.


The GenBank accession number for the 16S rRNA gene sequence of strain JR3T is PP065615 (https://www.ncbi.nlm.nih.gov/nuccore/PP065615) and for the genome sequence of strain JR3T is CP141832 (https://www.ncbi.nlm.nih.gov/nuccore/ CP141832.1).
 


The GenBank accession number for the 16S rRNA gene sequence of strain JR3T is PP065615 (https://www.ncbi.nlm.nih.gov/nuccore/PP065615) and for the genome sequence of strain JR3T is CP141832 (https://www.ncbi.nlm.nih.gov/nuccore/ CP141832.1).
 


strain names ['JR3T']
orgname ['Lacrimispora sinapis ']
accessions []
strain names ['JR3T']
BREAK1


In [8]:
#optional write description to a file
#print(combined_description)
file = open(alldescriptions, "w")
file.writelines(combined_description)
file.close()

In [9]:
pd.set_option('max_colwidth', None)
pub_df['Strains'] = [', '.join(map(str, l)) for l in pub_df['Strains']]
#pub_df['Strains'] = pub_df['Strains'].astype(str) 
pub_df['Strains'] = pub_df['Strains'].astype(pd.StringDtype())
pub_df['Strains'] = pub_df['Strains'].str.replace(',', ', ')
#pub_df
pub_df = pub_df.drop_duplicates(subset='PublishedName', keep="first")
#pub_df
#try drop duplicate accessions here
pub_df.explode(['PublishedName']).reset_index(drop=True)
pub_df

,PublishedName,Accessions,Strains,Authority,DOI,URL
0,[],[],,Lee et al. 2025,https://doi.org/10.1099/ijsem.0.006663,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006663%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C017a70c948af4333a1c808dd4ce97ee5%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638751288089330574%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C40000%7C%7C%7C&sdata=%2F9Mf43NMpFvAJ13NJ3KKsnI9C%2BRGKhXPDs0NPu7apYs%3D&reserved=0
1,[Thalassobellus suaedae ],"[OR229715, OR652609, OR229716, OR652608, CP134536, CP134537]","HL-DH10T, KCCM 90512T, JCM 36598T, HL-DH14, KCCM 90513",Lee et al. 2025,https://doi.org/10.1099/ijsem.0.006663,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006663%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C017a70c948af4333a1c808dd4ce97ee5%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638751288089330574%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C40000%7C%7C%7C&sdata=%2F9Mf43NMpFvAJ13NJ3KKsnI9C%2BRGKhXPDs0NPu7apYs%3D&reserved=0
4,[Methanobacterium aridiramus ],"[MK979366, CP040735]","CWC-01T, BCRC AR10053T, NBRC 113991T",Lee et al. 2025,https://doi.org/10.1099/ijsem.0.006658,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006658%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C017a70c948af4333a1c808dd4ce97ee5%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638751288089383537%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C40000%7C%7C%7C&sdata=n6Mm1qXs2TjrrOYlH1jFldblC5%2Bdu%2BbqcDRUZr1aCE4%3D&reserved=0
8,[Minisyncoccus archaeiphilus ],"[AP038758, LC847185]","PMX.108T, JCM 39522T",Nakajima et al. 2025,https://doi.org/10.5281/zenodo.10140531,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006668%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C017a70c948af4333a1c808dd4ce97ee5%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638751288089563102%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C40000%7C%7C%7C&sdata=3X9B3NLCY%2BU%2FdZOHG5rj0eHv4ShcDUe18vhPlUBhrAg%3D&reserved=0
18,[Afipia dichlorophenoxyacetatis ],"[AP029250, AP029255, LC770309, LC770310]","DD3T, MAFF 311804T, ICMP 25015T, MAFF 311814, ICMP 25016",Sawada et al. 2025,https://doi.org/10.1099/ijsem.0.006672,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006672%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C017a70c948af4333a1c808dd4ce97ee5%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638751288089597233%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C40000%7C%7C%7C&sdata=ASJ4qJ6s9rIYcPAJEuujidW%2BW8I65RB%2B0SZnXx5AKxE%3D&reserved=0
20,[Oceanobacter antarcticus ],"[PP494898, JBBKTX000000000]","wDCs-4T, MCCC 1A20726T, KCTC 8314T",Zhang et al. 2025,https://doi.org/10.1099/ijsem.0.006676,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006676%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C017a70c948af4333a1c808dd4ce97ee5%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638751288089633623%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C40000%7C%7C%7C&sdata=93hPxOFV1RNJzuGaS6UBfbT

In [ ]:
#pub_df['Strains'] = pub_df['Strains'].astype(pd.StringDtype())
#pub_df['Strains'] = pub_df['Strains'].str.replace(',', ', ')
#pub_df

In [10]:
pub2_df = pub_df.explode(['Accessions']).reset_index(drop=True)
#pub2_df
pub4_df = pub2_df.explode(['PublishedName']).reset_index(drop=True)
pub4_df.rename(columns={'Accessions' : 'accession'}, inplace=True)
#pub4_df = pub4_df.dropna()
#pub4_df = pub4_df.drop_duplicates(subset='accession', keep="first")
pub4_df=pub4_df[pub4_df['accession'].isnull() | ~pub4_df[pub4_df['accession'].notnull()].duplicated(subset='accession',keep='first')]
#pub4_df

In [11]:
df_unique= pub4_df.drop_duplicates(["accession"], keep="first")
#df_unique = df_unique.dropna()
#df_unique

### Create a dataframe of unique accessions and look up NCBI taxonomy information of each accession with srcchk

In [ ]:
#df_unique.dtypes

In [12]:
df_unique['accession'] = df_unique['accession'].astype('str') 

In [13]:
with open('acclist', 'w') as f:
    for text in df_unique['accession'].tolist():
        f.write(text + '\n')

In [14]:
os.system("/netopt/ncbi_tools64/bin/srcchk -i acclist -f taxname,taxid,strain -o acclist.taxdata")


Error: Malformatted ID "nan"


0

In [15]:
taxdata_file_name = (r'acclist.taxdata')    
srcchk_df = pd.read_csv(taxdata_file_name, sep='\t', index_col=None, low_memory=False)
srcchk_df.drop(columns=['Unnamed: 4'], inplace=True)
srcchk_df.rename(columns={'organism' : 'NCBIname'}, inplace=True)
srcchk_df['accession'] = srcchk_df['accession'].astype(str).replace('\.\d+', '', regex=True).astype(str)
srcchk_df = srcchk_df.dropna()
srcchk_df 
#many need to add dropna here
#If /strain is not populated on the NCBI accession, we lose the NCBI srcchk info here

,accession,NCBIname,taxid,strain
1,OR229715,Flavobacteriaceae bacterium,1871037.0,HL-DH10
2,OR652609,Flavobacteriaceae bacterium,1871037.0,HL-DH10
3,OR229716,Flavobacteriaceae bacterium,1871037.0,HL-DH14
4,OR652608,Flavobacteriaceae bacterium HL-DH14,3074125.0,HL-DH14
5,CP134536,Flavobacteriaceae bacterium HL-DH10,3074124.0,HL-DH10
6,CP134537,Flavobacteriaceae bacterium HL-DH14,3074125.0,HL-DH14
7,MK979366,Methanobacterium sp.,2164.0,CWC-01
8,CP040735,Methanobacterium sp. CWC-01,2584467.0,CWC-01
9,AP038758,Candidatus Minisyncoccus archaeophilus,3238481.0,PMX.108
10,LC847185,Candidatus Minisyncoccus archaeophilus,3238481.0,PMX.108


### Combine dataframes into one

In [16]:
combine_df=pd.merge(left=pub4_df, right=srcchk_df, left_on='accession', right_on='accession', how = 'outer')
combine_df = combine_df[['PublishedName', 'NCBIname', 'Strains', 'accession', 'strain', 'Authority', 'taxid', 'DOI', 'URL' ]]
combine_df

,PublishedName,NCBIname,Strains,accession,strain,Authority,taxid,DOI,URL
0,NaN,NaN,,NaN,NaN,Lee et al. 2025,NaN,https://doi.org/10.1099/ijsem.0.006663,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006663%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C017a70c948af4333a1c808dd4ce97ee5%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638751288089330574%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C40000%7C%7C%7C&sdata=%2F9Mf43NMpFvAJ13NJ3KKsnI9C%2BRGKhXPDs0NPu7apYs%3D&reserved=0
1,Thalassobellus suaedae,Flavobacteriaceae bacterium,"HL-DH10T, KCCM 90512T, JCM 36598T, HL-DH14, KCCM 90513",OR229715,HL-DH10,Lee et al. 2025,1871037.0,https://doi.org/10.1099/ijsem.0.006663,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006663%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C017a70c948af4333a1c808dd4ce97ee5%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638751288089330574%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C40000%7C%7C%7C&sdata=%2F9Mf43NMpFvAJ13NJ3KKsnI9C%2BRGKhXPDs0NPu7apYs%3D&reserved=0
2,Thalassobellus suaedae,Flavobacteriaceae bacterium,"HL-DH10T, KCCM 90512T, JCM 36598T, HL-DH14, KCCM 90513",OR652609,HL-DH10,Lee et al. 2025,1871037.0,https://doi.org/10.1099/ijsem.0.006663,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006663%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C017a70c948af4333a1c808dd4ce97ee5%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638751288089330574%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C40000%7C%7C%7C&sdata=%2F9Mf43NMpFvAJ13NJ3KKsnI9C%2BRGKhXPDs0NPu7apYs%3D&reserved=0
3,Thalassobellus suaedae,Flavobacteriaceae bacterium,"HL-DH10T, KCCM 90512T, JCM 36598T, HL-DH14, KCCM 90513",OR229716,HL-DH14,Lee et al. 2025,1871037.0,https://doi.org/10.1099/ijsem.0.006663,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006663%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C017a70c948af4333a1c808dd4ce97ee5%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638751288089330574%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C40000%7C%7C%7C&sdata=%2F9Mf43NMpFvAJ13NJ3KKsnI9C%2BRGKhXPDs0NPu7apYs%3D&reserved=0
4,Thalassobellus suaedae,Flavobacteriaceae bacterium HL-DH14,"HL-DH10T, KCCM 90512T, JCM 36598T, HL-DH14, KCCM 90513",OR652608,HL-DH14,Lee et al. 2025,3074125.0,https://doi.org/10.1099/ijsem.0.006663,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006663%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C017a70c948af4333a1c808dd4ce97ee5%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638751288089330574%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C40000%7C%7C%7C&sdata=%2F9Mf43NMpFvAJ13NJ3KKsnI9C%2BRGKhXPDs0NPu7apYs%3D&reserved=0
5,Thalassobellus suaedae,Flavobacteriaceae bacterium HL-DH10,"HL-DH10T, KCCM 90512T, JCM 36598T, HL-DH14, KCCM 90513",CP134536,HL-DH10,Lee et al. 2025,3074124.0,https://doi.org/10.1099/ijsem.0.006663,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006663%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C017a70c948af4333a1c808dd4ce97ee5%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C6387512880893305

In [17]:
def highlight_rows(row):
    ijsemvalue = row.loc['PublishedName']
    ncbivalue = row.loc['NCBIname']
    if ijsemvalue != ncbivalue:
        color = '#FFB3BA' # Red
    elif ijsemvalue == ncbivalue:
        color = '#BAFFC9' # Green
    return ['background-color: {}'.format(color) for r in row]

new_df = combine_df.style.apply(highlight_rows, axis=1, subset=['PublishedName', 'NCBIname'])
new_df

,PublishedName,NCBIname,Strains,accession,strain,Authority,taxid,DOI,URL
0,nan,nan,,nan,nan,Lee et al. 2025,nan,https://doi.org/10.1099/ijsem.0.006663,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006663%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C017a70c948af4333a1c808dd4ce97ee5%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638751288089330574%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C40000%7C%7C%7C&sdata=%2F9Mf43NMpFvAJ13NJ3KKsnI9C%2BRGKhXPDs0NPu7apYs%3D&reserved=0
1,Thalassobellus suaedae,Flavobacteriaceae bacterium,"HL-DH10T, KCCM 90512T, JCM 36598T, HL-DH14, KCCM 90513",OR229715,HL-DH10,Lee et al. 2025,1871037.000000,https://doi.org/10.1099/ijsem.0.006663,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006663%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C017a70c948af4333a1c808dd4ce97ee5%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638751288089330574%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C40000%7C%7C%7C&sdata=%2F9Mf43NMpFvAJ13NJ3KKsnI9C%2BRGKhXPDs0NPu7apYs%3D&reserved=0
2,Thalassobellus suaedae,Flavobacteriaceae bacterium,"HL-DH10T, KCCM 90512T, JCM 36598T, HL-DH14, KCCM 90513",OR652609,HL-DH10,Lee et al. 2025,1871037.000000,https://doi.org/10.1099/ijsem.0.006663,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006663%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C017a70c948af4333a1c808dd4ce97ee5%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638751288089330574%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C40000%7C%7C%7C&sdata=%2F9Mf43NMpFvAJ13NJ3KKsnI9C%2BRGKhXPDs0NPu7apYs%3D&reserved=0
3,Thalassobellus suaedae,Flavobacteriaceae bacterium,"HL-DH10T, KCCM 90512T, JCM 36598T, HL-DH14, KCCM 90513",OR229716,HL-DH14,Lee et al. 2025,1871037.000000,https://doi.org/10.1099/ijsem.0.006663,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006663%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C017a70c948af4333a1c808dd4ce97ee5%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638751288089330574%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C40000%7C%7C%7C&sdata=%2F9Mf43NMpFvAJ13NJ3KKsnI9C%2BRGKhXPDs0NPu7apYs%3D&reserved=0
4,Thalassobellus suaedae,Flavobacteriaceae bacterium HL-DH14,"HL-DH10T, KCCM 90512T, JCM 36598T, HL-DH14, KCCM 90513",OR652608,HL-DH14,Lee et al. 2025,3074125.000000,https://doi.org/10.1099/ijsem.0.006663,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006663%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C017a70c948af4333a1c808dd4ce97ee5%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638751288089330574%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C40000%7C%7C%7C&sdata=%2F9Mf43NMpFvAJ13NJ3KKsnI9C%2BRGKhXPDs0NPu7apYs%3D&reserved=0
5,Thalassobellus suaedae,Flavobacteriaceae bacterium HL-DH10,"HL-DH10T, KCCM 90512T, JCM 36598T, HL-DH14, KCCM 90513",CP134536,HL-DH10,Lee et al. 2025,3074124.000000,https://doi.org/10.1099/ijsem.0.006663,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006663%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C017a70c948af4333a1c808dd4ce97ee5%7C14b77578977342d58507251ca2dc2b06%7

### write output to excel

In [18]:
new_df.to_excel(output, engine='xlsxwriter', index = False, na_rep = '') 

In [ ]:
combine_df.dtypes